<a href="https://colab.research.google.com/github/ratulbempu/ApneBoot-Clinical-Data-Analysis/blob/main/Zugmed%20-%20AB%20Data%20Analysis%20Code%20RNv12%20221009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

22-07-23 - I updated it so it loads logs and creates df_s and df_d faster. Also includes Log Source File. Works for Zug, need to check for nonin. 
Actual and Preset are slightly different because:
- Preset mode number shows up at the start of the apnea, Stim shows at start of stim +1 (instead of 5s it looks like 6 sec+1. This may be a rounding issue. Would be nice if they were on the same line but ok. 
- This rounding issue may also cause different apnea stimulation lengths by a second or two.


22-09-03 - 
- Added code to convert table to pdf

22-09-10 -
- Fix Smoothing Issue
- Add Thresholds 

22-19-18
Todo:
- X make individual plots ouput to pdf
- X - Good Enough - make summary table cleaner
  - reduce number of columns
  - increase overall font size
- X test with custom thresholds
- test and fix for nonin
- combine relevent pdfs into a report.
- test with Clinical Data



## **Initialize**

### Set Folder Name

In [4]:
folder_name = "221018_Nonin_BAB20E1024_Yasinmalik"
folder_name = ""

data_path = "/content/drive/MyDrive/AB_Rental_Trail_Sample_units details/" + folder_name

data_path = "/content/drive/MyDrive/Improve World Health/Document Control/NBD - New Business Development/NBD028 - ApneBoot/Clinical Data/St. John's Data/st. John's data_For Analysis/intervention/Baby of Geetha/Geetha"

Data_Cleanup_SP02_LowCutOff = 60  #If the SPO2 value at the time of APnea is below 60, it must be a bad data point. 
Data_Cleanup_HR_LowCutOff = 60
OEMType = " "

time: 1.81 ms (started: 2023-03-01 08:20:11 +00:00)


### Import and Initialize Folders

In [5]:
import fileinput
import glob
import pandas as pd
import time
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import os
import linecache
from pathlib import Path
pd.options.mode.chained_assignment = None  # default='warn'
!pip install ipython-autotime
%load_ext autotime

%matplotlib inline
print ("Imports done")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Imports done
time: 3.95 s (started: 2023-03-01 08:20:12 +00:00)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

# path = "/content/drive/MyDrive/AB_Data_trial_and_rental_unit/" + folder_name

os.chdir(data_path)
!mkdir Analysis_Files


analysis_path = data_path + "/Analysis_Files/"

os.chdir(analysis_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 3.42 s (started: 2023-03-01 08:20:16 +00:00)


## **Convert Files to Data Frames**

### Headers for Different Logger Versions

In [7]:
def decideDataLoggerVersion (df):
  # Read the third line's header and decide which data logger version is used
  # Assumes that the third line is unique for each data logger version
  # I need to check with Sourabh if the third line correlates with versions for the St. John's studies.
  global OEMType
  raw_logfile_data_table_headers = ["default"]
  settings_table_headers = [""]
  data_table_headers = [""]
  settings_columns_rename_dict = [""]
  settings_columns_to_drop = [""]
  #header = ""
  version = "unknown data code version"

  ZUG_DATA_HEADER = "dd/mm/yy,hh:mm:ss,SPO2,HR,PI,Status,Pulse-ox mode,Threshold Met,Stimulator Error,Stimulator ON/OFF,Power Mode,Battery voltage;"
  NONIN_DATA_HEADER = "dd/mm/yy,hh:mm:ss,SPO2,HR,Status,Pulse-ox mode,Threshold Met,Stimulator Error,Stimulator ON/OFF,Power Mode,Probe Off patient,Battery voltage"


  # file = open (file_name)
  # header = linecache.getline(file_name, 3)
  header = df.iat[2,0]
  print (header)

  #print ("Header is: " + header)
  if header == ZUG_DATA_HEADER:
    version = "ZUG Circa April 2022"
    print ("Detected Zug")
  elif (header == NONIN_DATA_HEADER):
    version = "NONIN Circa April 2022"
    print ("Detected Nonin")
  else:
    print ("Not able to Detect Nonin Nor Zug")
  
  print (version)

  if version == "ZUG Circa April 2022":
    OEMType = "ZUG Circa April 2022"
        ## Zug Head
    raw_logfile_data_table_headers = [#"Settings Index",
                          #"Data Date-Time",
                           "Data Date",
             "Data Time",
            "SPO2",
            "HR",
            "PI",
            "Status",
            "Pulse-ox mode",
            "Threshold Met",
            "Stimulator Error",
            "Stimulator ON/OFF",
            "Power Mode",
        #     "Probe Off patient", - July 22, 2022 - RN checked this with Sourabh - this value was not being recorded although it is in the header
            "Battery voltage",
            #"Settings Index" # This is added at the end and is blank until filled
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "SPO2",
                "HR",
                "PI",
                "Status",
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",   
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
#                "Settings Index": int,
                "Data Date-Time": 'datetime64[ns]',
                "Data Date": 'datetime64[ns]',
                "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
                "PI": float,
                "Status": int,
                "Pulse-ox mode":int,
                "Threshold Met":int,
                "Stimulator Error":int,
                "Stimulator ON/OFF":int,
                "Power Mode":int,
            #     "Probe Off patient":int,
                "Battery voltage": float
                #,"Settings Index": int
                }

    settings_convert_dict = {
#                "Settings Index": int,
                "Settings Date-Time": str,
                "Settings Date": str,
                'Settings Time':str,
                'Preset mode':int,
                'SPO2 Threshold':int,
                "HR Threshold":int,
                "AND/OR":int,
                "Duration":int,
                "Stimulator Intensity":int,
                "Alarm intensity":str}
    settings_columns_rename_dict = {
             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            "SPO2":"Preset mode",
            "HR":"SPO2 Threshold",
            "PI":"HR Threshold",
            "Status":"AND/OR",
            "Pulse-ox mode":"Duration",
            "Threshold Met":"Stimulator Intensity",
            "Stimulator Error":"Alarm intensity"
        #     "Stimulator ON/OFF",
        #     "Power Mode",
        # #     "Probe Off patient",
        #     "Battery voltage",
        #     "Settings Index" # This is added at the end and is blank until filled
        }
    
    settings_columns_to_drop = ["Stimulator ON/OFF",
           "Power Mode",
            "Battery voltage"]

  if version == "NONIN Circa April 2022":
    OEMType = "NONIN Circa April 2022"
    raw_logfile_data_table_headers = [#"Settings Index",
                          #"Data Date-Time",
                           "Data Date",
             "Data Time",
            "SPO2",
            "HR",
         #   "PI",
            "Status",
            "Pulse-ox mode",
            "Threshold Met",
            "Stimulator Error",
            "Stimulator ON/OFF",
            "Power Mode",
        #     "Probe Off patient", - July 22, 2022 - RN checked this with Sourabh - this value was not being recorded although it is in the header
            "Battery voltage",
            "Extra Column"
            #"Settings Index" # This is added at the end and is blank until filled
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "SPO2",
                "HR",
          #      "PI",
                "Status",
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
                #"Settings Index": int,
                "Data Date-Time": 'datetime64[ns]',
                "Data Date": 'datetime64[ns]',
                "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
           #     "PI": float,
                "Status": int,
                "Pulse-ox mode":int,
                "Threshold Met":int,
                "Stimulator Error":int,
                "Stimulator ON/OFF":int,
                "Power Mode":int,
            #     "Probe Off patient":int,
                "Battery voltage": float}
                #"Settings Index": int}

    settings_convert_dict = {
                "Settings Date-Time": 'datetime64[ns]',
                "Settings Date": 'datetime64[ns]',
                'Settings Time':'datetime64[ns]',
                'Preset mode':int,
                'SPO2 Threshold':int,
                "HR Threshold":int,
                "AND/OR":int,
                "Duration":int,
                "Stimulator Intensity":int,
                "Alarm intensity":str,
                #"Settings Index": int
                }
    settings_columns_rename_dict = {

             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            "SPO2":"Preset mode",
            "HR":"SPO2 Threshold",
            "Status":"HR Threshold",
            "Pulse-ox mode":"AND/OR",
            "Threshold Met":"Duration",
            "Stimulator Error":"Stimulator Intensity",
            "Stimulator ON/OFF":"Alarm intensity"
        #     "Stimulator ON/OFF",
        #     "Power Mode",
        # #     "Probe Off patient",
        #     "Battery voltage",
        #     "Settings Index" # This is added at the end and is blank until filled
        }
    settings_columns_to_drop = [
           "Power Mode",
            "Battery voltage",
            "Extra Column"]

  return raw_logfile_data_table_headers, settings_table_headers, data_table_headers, settings_columns_rename_dict, settings_columns_to_drop, settings_convert_dict, data_convert_dict

time: 8.2 ms (started: 2023-03-01 08:20:20 +00:00)


### Compile & Split Log Files to df_d and df_s

In [8]:
# Reference here: https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
def compileLogFiles (path):
  
  print (path)
  os.chdir(path)
  file_list = sorted(glob.glob("LOG*"))
  number_of_files = len(file_list)
  print("Number of Files:" , len(file_list))
  print (file_list)
  li = []

  for file_name in file_list:
    # This could work faster potentially, but then the data logger identifier code for finding the zug/nonin code string has to be changed
    # df = pd.read_csv(file_name, header=None, sep='\n', names=list(range(12)))
  
    df = pd.read_csv(file_name, header=None, sep='\n')
    #df = pd.read_csv(file_name, header=None, sep=',')
    df['Source File'] = file_name
    li.append (df)
  
  frame = pd.concat(li, axis=0, ignore_index=True)
  return frame


time: 3.09 ms (started: 2023-03-01 08:20:20 +00:00)


In [9]:
def splitToDataAndSettingsDF (df_LOGs):
  
  raw_logfile_data_table_headers, settings_table_headers, data_table_headers, settings_columns_rename_dict, settings_columns_to_drop, settings_convert_dict, data_convert_dict = decideDataLoggerVersion (df_LOGs)    

  df_raw = df_LOGs[0].str.split(',', expand=True)
  #display (df_raw)
   
  df_raw.columns = raw_logfile_data_table_headers
  df_raw["Source File"] = df_LOGs ["Source File"]

  # Extract Settings
  df_raw_shift = df_raw.shift(periods=-1)
  df_s = df_raw_shift[df_raw.iloc[:,2]=="Preset mode"]

  # Delete settings and settings headers
  df_raw_shift = df_raw.shift(periods=1)                                        #shift down one row as a way to find one line below the settings header
  rows_below_header = df_raw[df_raw_shift.iloc[:,2]=="Preset mode"]
  df_raw = df_raw.drop(rows_below_header.index)                                 #Drop all the setttings data lines 
  rows_with_header = df_raw[df_raw.iloc[:,2]=="Preset mode"]
  df_raw = df_raw.drop (rows_with_header.index)                                 #Drops all the settings headers
  rows_with_header = df_raw[df_raw.iloc[:,2]=="SPO2"]
  df_raw = df_raw.drop (rows_with_header.index)                                 #Drops all the DATA headers
  
  # Format Settings Files
  df_s.drop(settings_columns_to_drop, axis=1, inplace=True)                     # Drop extra columns
  df_s.rename(columns=settings_columns_rename_dict, inplace=True)
  df_s['Alarm intensity'].apply(lambda v: v.rstrip(';'))
  
  #What's left of df_raw is now df_d for data
  df_d = df_raw
  if (OEMType == "NONIN Circa April 2022"):
    df_d.drop (columns = "Extra Column", axis = 1, inplace=True)
    #df_s.drop (columns = "Extra Column", axis = 1, inplace=True)
  
  # Insert Data-Time Column
  df_s.insert(loc=0, column="Settings Date-Time", value=pd.to_datetime(df_s["Settings Date"] + ' ' + df_s["Settings Time"] ,format='%d/%m/%y %H:%M:%S'))
  df_d.insert(loc=0, column="Data Date-Time", value=pd.to_datetime(df_d["Data Date"] + ' ' + df_d["Data Time"] ,format='%d/%m/%y %H:%M:%S'))
  ####time data 25/08/22 18:54:26 doesn't match format specified
  
  #Cleanup NaNs
  df_d = df_d.fillna(0)
  df_s = df_s.fillna(0)

  #Convert to best datatypes
  df_s["Settings Date"]=pd.to_datetime(df_s["Settings Date"],format='%d/%m/%y')
  df_s["Settings Time"]=pd.to_datetime(df_s["Settings Time"],format='%H:%M:%S')#.dt.time
  df_d["Data Date"]=pd.to_datetime(df_d["Data Date"],format='%d/%m/%y')
  df_d["Data Time"]=pd.to_datetime(df_d["Data Time"],format='%H:%M:%S')#.dt.time

  
  df_s=df_s.astype(settings_convert_dict, errors='raise')
  df_d=df_d.astype(data_convert_dict, errors='raise')

  #df_d = ResolveMidnightDateIssue (df_d) 

#  df_d["Data Date-Time"]=pd.to_datetime(df_d["Data Date-Time"],format='%d/%m/%y %H:%M:%S')

  # For Settings Table delete the confusing default numbers left in place when "preset" condition is on.
  df_s = DeleteThresholdsForRowsUsingPresetConditions(df_s)
  
  
  df_s, df_d = addSettingsIndex (df_s,df_d)
  
  return df_s, df_d

time: 7.3 ms (started: 2023-03-01 08:20:20 +00:00)


In [10]:
# It was observed with Zug that for the first ~30 seconds after midnight, the date does not change. This needs to be fixed or "smoothed" with the code below.
#import calendar

#### This function is not finished.
def ResolveMidnightDateIssue (df_above):
  print ("in the smooth")
  df_diff = pd.DataFrame ()
  df_below = pd.DataFrame ()

  df_below["Data Date-Time"] = df_above["Data Date-Time"].shift (periods=1, fill_value=0)
  #df_below["Data Date"] = df_above["Data Date"].shift (periods=1, fill_value=0)
  df_below ["Date Diff"] = df_above["Data Date-Time"] < df_below["Data Date-Time"]
  df_below ["Date Diff Num"] = (df_above["Data Date-Time"] - df_below["Data Date-Time"]).dt.total_seconds()
  print (df_below ["Date Diff Num"].dtype)
  

  # display(df_above)
  # display(df_below)
  # display (df_below[df_below["Date Diff"]==1])
  df_midnight_bounds = df_below[df_below["Date Diff Num"].abs()>1000]
  #display (df_midnight_bounds)
  # for i in df_midnight_bounds.index:
  #   print (i)



  #display (df_below[df_below["Date Diff Num"]!=1])
  
  print ("Sum is: " + df_below["Date Diff"].astype(int).sum().astype(str))
  
  return df_above




time: 3.5 ms (started: 2023-03-01 08:20:20 +00:00)


In [11]:
def addSettingsIndex (df_s,df_d):
  #df_s["Settings Index"] = np.arange(len(df_s))
  df_s.insert(loc=0, column="Settings Index", value=np.arange(len(df_s))+1)
  df_d = pd.merge_asof (df_d,df_s["Settings Index"],left_index=True, right_index=True)
  SettingsIndexCol = df_d.pop('Settings Index')
  df_d.insert(0, 'Settings Index', SettingsIndexCol)
  
  return df_s, df_d

time: 2.44 ms (started: 2023-03-01 08:20:20 +00:00)


In [12]:
def DeleteThresholdsForRowsUsingPresetConditions (settings_table):
    # display(settings_table)
    for ind in settings_table.index:
        if settings_table["Preset mode"][ind] == 1:
            settings_table["SPO2 Threshold"][ind] = np.NaN
            settings_table["HR Threshold"][ind] = np.NaN
            settings_table["AND/OR"][ind] = np.NaN
            settings_table["Duration"][ind] = np.NaN
                       #Preset mode	SPO2 Threshold	HR Threshold	AND/OR	Duration
    return settings_table


time: 2.5 ms (started: 2023-03-01 08:20:20 +00:00)


## **Analyze Thresholds**

### Analyze Against Custom Thresholds
This function returns what is expected in the custom mode the user selected. It should match with actual stimulations in custom mode.

In [13]:
def AnalyzeWithCustomThresholds (df):
  is_valid = ((df["SPO2"]>Data_Cleanup_SP02_LowCutOff) & (df["HR"]>Data_Cleanup_HR_LowCutOff))
  is_below_threshold_AND = is_valid & ((df["SPO2"]< df["SPO2 Threshold"]) & (df["HR"]< df["HR Threshold"]))
  is_below_threshold_OR = is_valid & ((df["SPO2"]< df["SPO2 Threshold"]) | (df["HR"]< df["HR Threshold"]))
  is_below_threshold_result = (df["AND/OR"]==0)*is_below_threshold_OR + (df["AND/OR"]==1)*is_below_threshold_AND
  # is_below_threshold_result = is_below_threshold_result.mask (is_below_threshold_result <= df["Duration"],0) #This statement removes any apnea durations that are below the threshold at that row where the apnea started.
  df_unmasked = calcDurationsCrossedThreshold(pd.DataFrame(is_below_threshold_result))

  df[Duration_Custom] = df_unmasked.mask(df_unmasked <= df["Duration"],0)


  return df

time: 3.57 ms (started: 2023-03-01 08:20:20 +00:00)


### Analyze With Preset Thresholds
This function is useful to see how the system would have performed if it were operating in preset mode.

In [14]:
SPO2_threshold_AND_condition = 85
HR_threshold_AND_condition = 100
SP02_threshold_OR_condition = 75
HR_threshold_OR_condition  = 90
Preset_Mode_Duration = 5

def AnalyzeWithPresetThresholds (df):
  

  is_valid = ((df["SPO2"]>Data_Cleanup_SP02_LowCutOff) & (df["HR"]>Data_Cleanup_HR_LowCutOff))
  is_below_threshold_AND = is_valid & ((df["SPO2"] < SPO2_threshold_AND_condition) & (df["HR"]< HR_threshold_AND_condition))
  is_below_threshold_OR = is_valid & ((df["SPO2"]< SP02_threshold_OR_condition) | (df["HR"]< HR_threshold_OR_condition))
  
  # is_below_threshold_result = ((df["AND/OR"]==0))&(is_below_threshold_OR) | ((df["AND/OR"]==1) &is_below_threshold_AND)
  is_below_threshold_result = is_below_threshold_OR | is_below_threshold_AND
  #print ("Sum is:" + str(sum(is_below_threshold_result)))
  display (is_below_threshold_result)
  # is_below_threshold_result = is_below_threshold_result.mask (is_below_threshold_result <= Preset_Mode_Duration,0) #This statement removes any apnea durations that are below the threshold at that row where the apnea started.
  # print ("Sum is:" + str(sum(is_below_threshold_result)))

  # print ("Total preset results are: " + str(is_below_threshold_result.sum()))
  df_unmasked = calcDurationsCrossedThreshold(pd.DataFrame(is_below_threshold_result))
  df[Duration_Preset] = df_unmasked.mask(df_unmasked <= Preset_Mode_Duration,0)

  #print ("Sum is:" + str(sum(df[Duration_Preset])))

  return df


time: 7.21 ms (started: 2023-03-01 08:20:20 +00:00)


### Analyze Actual Stimulations
This function looks at Stim Off/On field and counts the apnea durations (assuming SPO2 & HR are not 0). It does not consider the Preset/Custom settings at all. 

In [15]:
# Assumes it is being passed df_j
def AddNumActualStimulations (df):
  
  Valid_Stimulations = df["Stimulator ON/OFF"]*(df["SPO2"]>Data_Cleanup_SP02_LowCutOff)*(df["HR"]>Data_Cleanup_HR_LowCutOff)
  df[Duration_Actual] = calcDurationsCrossedThreshold(pd.DataFrame(Valid_Stimulations))
   
  ## New code (22-07-23) - I later realized that with the new apnea duration counter, we don't need to "Stimulation started column" as it easily puts the apnea duration time where the stimulation started. 
  # df["Stimulator ON/OFF with Valid SPO2/HR"] = df["Stimulator ON/OFF"]*(df["SPO2"]!=0)*(df["HR"]!=0) 

  # df_previousrow = df.shift(periods=1)
  # isStimulationStarted = (
  #                         (df["Stimulator ON/OFF with Valid SPO2/HR"]==1) & 
  #                         (df_previousrow["Stimulator ON/OFF with Valid SPO2/HR"] == 0) &
  #                         (df["SPO2"]!=0) & 
  #                         (df["HR"]!=0)
  #                         )  # This condition was added because there are some odd cases where the stimulator turns when HR/SPO2 are 0...

  #This gives 408 rows, but the one above gives 411 rows. There are likely some times then the O2 or HR jumps to zero mid apnea but jumps back up and the stimulator is on the whole time? 
  # OldisStimulationStarted = (
  #                         (df["Stimulator ON/OFF"]==1) & 
  #                         (df_previousrow["Stimulator ON/OFF"] == 0) &
  #                         (df["SPO2"]!=0) & 
  #                         (df["HR"]!=0))

  #Should be 408 Rows
  
  # df["Stimulation Triggered (Actual Stimulator ON Triggered)"] = isStimulationStarted
  # df["Actual - Stimulation Durations (s)"] = calcDurationsCrossedThreshold(pd.DataFrame(df["Stimulator ON/OFF with Valid SPO2/HR"]))
  return df

time: 2.21 ms (started: 2023-03-01 08:20:20 +00:00)


### Threshold Cross Duration
This takes an array of "threshold crossed" or "stimulation on" (0,0,0,1,1,1,1,0..etc) and returns an array with the time in the first place the threshold is cross i.e. (0,0,0,4,0,0,0,0...)

In [16]:
# Input: A df with 0s and 1s representing cross thresholds
# see this -  https://stackoverflow.com/questions/62615666/how-to-sum-variable-ranges-in-a-pandas-column-to-another-column
def calcDurationsCrossedThreshold (df):
  
  df = df.iloc[::-1]
  a = df.iloc[:,0]==0
  blocks = a.cumsum()
  
  m = blocks.duplicated(keep='last')
  
  df['Sum'] = df.iloc[:,0].groupby(blocks).cumsum().mask(m)
  df = df.iloc[::-1]
  df_return = df ['Sum'].fillna(0)
  return df_return

time: 3.62 ms (started: 2023-03-01 08:20:20 +00:00)


## **Generate Summary Tables**

In [17]:
def reportApneaCountBySetting (df,df_s):
  table = pd.pivot_table (df, values = [Duration_Actual, Duration_Preset, Duration_Custom],
                          index = "Settings Index",
                          aggfunc='sum')
  display (table)
  table = table.rename (columns = {Duration_Actual:SumDuration_Actual, Duration_Preset:SumDuration_Preset, Duration_Custom:SumDuration_Preset})
  df_s=df_s.join(table, on="Settings Index")
  

  #Count function counts zeros too, so need to replace zeros with NAs for this to work.
  df_nan = df.mask(df==0) # This will make all zeros into Nan
  table = pd.pivot_table (df_nan, values = [Duration_Actual, Duration_Preset, Duration_Custom],
                          index = "Settings Index",
                          aggfunc='count')
  
  table = table.rename (columns = {Duration_Actual:Count_Actual, Duration_Preset:Count_Preset, Duration_Custom:Count_Custom})
  df_s=df_s.join(table, on="Settings Index")
  display (df_s)
  df_s.to_excel (folder_name + " - Summary of Events.xlsx")
  dataframe_to_pdf(df_s, folder_name + " - Apnea Count By Setting")
  return






time: 4.59 ms (started: 2023-03-01 08:20:20 +00:00)


In [18]:
def reportIndividualActualApneas (df):
  table_A = df[df[Duration_Actual]!=0]
  table_P = df[df[Duration_Preset]!=0]
  table_C = df[df[Duration_Custom]!=0]
  table_A_P = df[ (df["Preset mode"]==1 & (df[Duration_Actual]!=df[Duration_Preset]))]
  table_A_C = df[ (df["Preset mode"]==0 & (df[Duration_Actual]!=df[Duration_Custom]))]
                   
  
  with pd.ExcelWriter(folder_name + " - Individual Events.xlsx") as writer:
    table_A.to_excel (writer, sheet_name="Actual")
    table_P.to_excel (writer, sheet_name="Preset")
    table_C.to_excel (writer, sheet_name="Custom")
    table_A_P.to_excel (writer, sheet_name="Actual-Preset Discrepancy")
    table_A_C.to_excel (writer, sheet_name="Actual-Custom Discrepancey")

  dataframe_to_pdf(table_A, folder_name + " - Individual Events (Actual)")
  return



time: 5.59 ms (started: 2023-03-01 08:20:20 +00:00)


In [19]:
def convertDataframeToFigure (df):
  display (df)
  fig, ax =plt.subplots(figsize =(20,20))
  #fig, ax =plt.subplots(figsize=(len(df),len(df.columns)))
  ax.axis('tight')
  ax.axis('off')
  the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='center')
  return fig




time: 2.26 ms (started: 2023-03-01 08:20:20 +00:00)


## Print Tables to PDF
This code enables printing tables to pdf.

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import textwrap as twp

# #create column headers with long text and apply textwrap with \n after defined lenght
# columns=[twp.fill('this is a long column header text',25), twp.fill('...and this is an even longer column header text',25)]

# https://predictivehacks.com/?all-tips=save-a-pandas-dataframe-as-an-image
# pip install dataframe-image
# import pandas as pd
# import dataframe_image as dfi
 
# dfi.export(df, 'dataframe.png')

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis('tight')
    ax.axis('off')
    # df=df.to_string(line_width=5)
    # .str.wrap(5)
    col_headers=list(df.columns.str.wrap(15)) #df.columns#.map("str")
    #display (col_headers)
    #print ("Cols is " + len(df[0]))
    #display (df)
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=col_headers,
                        #colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    the_table.auto_set_column_width(col=list(range(len(df.columns)))) # Provide integer list of columns to adjust

    # the_table.auto_set_font_size(False)
    # the_table.set_fontsize(12)
    return fig
  

def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    display (df)
    # nh, nv = numpages
    # rows_per_page = len(df) // nh
    # cols_per_page = len(df.columns) // nv

    # nh, nv = numpages
    rows_per_page = 30
    cols_per_page = len(df.columns)
    nv = 1
    
    if len(df) % rows_per_page == 0: 
      nh = int(len(df) / rows_per_page)
    else:
      nh = int(len(df) / rows_per_page) +1
    
    print ('num cols is ' + str(len(df.columns)))
    print ('num rows is ' + str(len(df)) + "nv is " + str(nv))

    for i in range(0, nh):
        for j in range(0, nv):
            print ("i is " + str(i) + "and j is " + str(j))
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            display (page)
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            
            pdf.savefig(fig, bbox_inches='tight')
            plt.show ()
            plt.close()


time: 400 ms (started: 2023-03-01 08:20:20 +00:00)


## **Plots**

### Plot AB Data

In [21]:
def plotApneBootData (df,plot_title,x_axis_start,x_axis_end):
    #display (df)
    
    f, (ax, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [10, 1]})

    ax2 = df.plot(x="Data Date-Time", y="Status", rot=0,ax=ax2)
    
    plotSPO2AndHR(df,plot_title,x_axis_start,x_axis_end,ax)
       
    f.tight_layout()
    
    return ax




time: 3 ms (started: 2023-03-01 08:20:21 +00:00)


In [22]:
def plotSPO2AndHR (df,plot_title,x_axis_start,x_axis_end,ax):
  df["Stimulator ON/OFF Plot"] = df["Stimulator ON/OFF"]*20




  ax = df.plot(x="Data Date-Time", y="SPO2", 
                            figsize=(30,20),
                            title = plot_title,
                            ylim=(-5,150),
                            xlim= (x_axis_start, x_axis_end),
                            grid=True,
                            #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                            rot=0,
                            #kind='scatter',
                            color = 'red',
                            label = "SPO2",
                            linewidth=1, 
                            #s=2,
                            ax=ax)
    #ax.grid()
  df.plot(x="Data Date-Time", y="HR", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=True,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          color = 'blue',
                          label = "HR",
                          linewidth=1,
                          #s=2,
                          ax=ax)
  
  df.plot.line(x="Data Date-Time", y="Stimulator ON/OFF Plot", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=1 ,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          linewidth=4,
                          alpha=0.2,
               
                          #fmt ='+',
                          color = 'Orange',
                          label = "Stim ON",
                          #s=30,
                          ax=ax)
                          
  df_t = getThresholdTable (df)
  
  df_t.plot.line(x="Data Date-Time", y="SPO2 Threshold", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=1 ,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          color = 'DarkRed',
                          label = "SPO2 Threshold",
                          linewidth=3,
                          linestyle='dashed',
                          alpha=0.2,
                          #s=2,
                          ax=ax
                          )
  
  df_t.plot.line(x="Data Date-Time", y="HR Threshold", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=1 ,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          color = 'DarkBlue',
                          label = "HR Threshold",
                          linewidth=3,
                          linestyle='dotted',
                          alpha=0.2,
                          #s=2,
                          ax=ax
                          )
  
  x_offset = (x_axis_end - x_axis_start)/10
  df_a = df[(df[Duration_Actual] > 0)]
  #df_a["Text X Position"] = df_a["Data Date-Time"][ind] - x_offset
  df_a=df_a.reset_index(drop=True)
  df_a["Arrow Offet"] = df_a["Data Date-Time"] - x_offset

  for ind in df_a.index:
    if df_a["Arrow Offet"][ind] <= x_axis_start:
      print (str(df_a["Arrow Offet"][ind]) + "is less than " + str(x_axis_start))
      df_a["Arrow Offet"][ind] = df_a["Arrow Offet"][ind] + x_offset*2
      print ("so we replace with: " + str(df_a["Arrow Offet"][ind]))
    #print (ind)
    ax.annotate("Event For " + df_a[Duration_Actual][ind].astype(int).astype(str) + 's\n' + df_a["Data Date-Time"][ind].strftime('%Y-%m-%d %X') + '\n SPO2:' + df_a["SPO2"][ind].astype(str) + " HR:" + df_a["HR"][ind].astype(str),
            xy = (df_a["Data Date-Time"][ind], df_a["SPO2"][ind]), 
            fontsize = 10, 
            xytext = (df_a["Data Date-Time"][ind] - x_offset, ind%4*15+20), 
            arrowprops = dict(facecolor = 'yellow'),
            color = 'black',
            backgroundcolor='white')
    
  #df.plot.annotate(df[Duration_Actual].str(), xy="Data Date-Time", y=100, , fontsize = 10, color = 'g')

  #### PRINT THresholds
  
  # ax.plot.line (x=df["Data Date-Time"], y=df_t["HR Threshold"], 
  #                         #figsize=(20,5),
  #                         #title = current_date,
  #                         #ylim=(-5,150),
  #                         #xlim= (x_axis_start, x_axis_end),
  #                         #grid=1 ,
  #                         #xticks=pd.date_range(x_axis_start, x_axis_end,24),
  #                         #rot=1,
  #                         #kind='scatter',
  #                         color = 'Orange',
  #                         label = "HR Threshold",
  #                         linewidth=1,
  #                         #s=2,
  #                         ax=ax
  #                         )



  ax.grid('on', which='major')
  return ax

time: 7.45 ms (started: 2023-03-01 08:20:21 +00:00)


### Get Threshold Table

In [23]:
def getThresholdTable (df):
  SPO2_threshold_AND_condition = 85
  HR_threshold_AND_condition = 100
  SP02_threshold_OR_condition = 75
  HR_threshold_OR_condition  = 90
  Preset_Mode_Duration = 5

  df_t = pd.DataFrame()
  #df_t = df
  # df_t ["HR Threshold"] = 60
  # df_t ["SPO2 Threshold"] = 64
  df_t ["Data Date-Time"] = df["Data Date-Time"]
  df_t ["HR Threshold"] = df["Preset mode"]*HR_threshold_OR_condition
  df_t ["SPO2 Threshold"] = df["Preset mode"]*SP02_threshold_OR_condition
  df_t["HR Threshold"][df_t["HR Threshold"]==0]=df["HR Threshold"][df_t["HR Threshold"]==0]
  df_t["SPO2 Threshold"][df_t["SPO2 Threshold"]==0]=df["SPO2 Threshold"][df_t["SPO2 Threshold"]==0]

  # df_t ["HR Custom"] = df["HR Threshold"]
  # df["SPO2 Threshold"].notna().astype(int)*df["SPO2 Threshold"]
  # display (df_t)

  #is_below_threshold_AND = is_valid & ((df["SPO2"] < SPO2_threshold_AND_condition) & (df["HR"]< HR_threshold_AND_condition))
  
  #print ("df_t is: ")
  #display (df_t)

  return df_t

time: 3.2 ms (started: 2023-03-01 08:20:21 +00:00)


### Plot by Day

In [24]:
# df_plot=df_j

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates


def plotByDay (df):
    locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
    formatter = mdates.ConciseDateFormatter(locator)

    df["Data Date"] = pd.to_datetime(df["Data Date"])
    date_list = np.unique(df["Data Date"])
    length = len(date_list)

    fname = folder_name + ' plot_by_day.pdf'
    with PdfPages(fname) as pdf:
        for i in range(length):
          current_date_64 = date_list[i]
          current_date=pd.to_datetime(current_date_64)
          
          # Create a dataframe of only one date
          df_current_date = df[(df["Data Date"] == current_date)]
          #display(df_current_date)
          
          # x axis start and end
          x_axis_start = current_date.replace(second = 1)
          #print(x_axis_start)
          x_axis_end = current_date.replace(hour = 23, minute=59, second = 59)
          
          
          ##### Sept 7 - Not sure if this below has to be commented or fixed
          # df_current_date["Data Time"] = pd.to_datetime(df_current_date["Data Time"].astype(str)).dt.time
          # #df_current_date["Data Time"] = pd.to_datetime(df_current_date["Data Time"])
          df_current_date["Data Time"] = df_current_date["Data Time"].dt.time
          df_current_date.set_index('Data Time')
          #display(df_current_date.dtypes)
          #display (df_current_date)
          
          plotApneBootData (df_current_date,current_date_64,x_axis_start,x_axis_end)
          pdf.savefig()
          plt.close()





time: 3.18 ms (started: 2023-03-01 08:20:21 +00:00)


### Plot By Hour

In [25]:
def plotByHour (df):
    locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
    formatter = mdates.ConciseDateFormatter(locator)

    df["Data Date"] = pd.to_datetime(df["Data Date"])
    df["Data Time"] = pd.to_datetime(df["Data Time"])
    df["Data Time-Hour"] = df["Data Date-Time"].dt.hour
    # display(df)
    # return
    date_list = np.unique(df["Data Date"])
    date_length = len(date_list)
    fname = folder_name + ' plot_by_hour.pdf'
    with PdfPages(fname) as pdf:

        firstPage = plt.figure(figsize=(11.69,8.27))
        firstPage.clf()
        txt = 'This is the title page'
        firstPage.text(0.5,0.5,txt, transform=firstPage.transFigure, size=24, ha="center")
        pdf.savefig()
        plt.close()




        for i in range(date_length):
          current_date_64 = date_list[i]
          current_date=pd.to_datetime(current_date_64)
          
          # Create a dataframe of only one date
          df_current_date = df[(df["Data Date"] == current_date)]
          #display(df_current_date)
      
          hour_list = np.unique(df_current_date["Data Time-Hour"])
          hour_length = len(hour_list)
          #print (hour_list)

          for j in range(hour_length):
              current_hour_64 = hour_list[j]
              #current_date=pd.to_datetime(current_date_64)
              df_current_hour = df_current_date[(df_current_date["Data Time-Hour"]==current_hour_64)]
              x_start = current_date + pd.offsets.Hour(current_hour_64)
              x_end = current_date + pd.offsets.Hour(current_hour_64+1)
              df_current_hour['Data Date-Time'].loc[df_current_hour.index[0]]
              # x_end = x_start + pd.offsets.Hour()
              
              # x_start = df_current_hour['Data Date-Time'].loc[df_current_hour.index[0]]
              # x_end = x_start + pd.offsets.Hour()
              title = str (current_date) + ' ' + str(x_start)
              plotApneBootData (df_current_hour,x_start,x_start,x_end)
              # plotApneBootData (df_current_hour,current_date,current_hour_64,current_hour_64+1)
              pdf.savefig()
              plt.close()


time: 4.5 ms (started: 2023-03-01 08:20:21 +00:00)


### SubPlot Grids of Apnea Events 

In [26]:
### Plot multiple subplots

def PlotApneaEventsGrid (df):
  
  start_offset = dt.timedelta (seconds=15)
  end_offset = dt.timedelta (seconds=45)
  num_rows = 2
  num_cols = 2
  num_subplots = num_rows * num_cols

  df_list_of_apnea_events = df[(df[Duration_Actual] != 0)]
  
  num_events = len(df_list_of_apnea_events)
  # print ("div " + str(num_events/num_subplots))
  # print ("div " + str(9/4))
  # print ("div " + str(np.ceil(9/4)))
  # #num_pages = int(num_events/num_subplots + 1)
  num_pages = int(np.ceil(num_events/num_subplots))
  #print ("number of pages is " + str (num_pages))
  #print ("number of events is " + str (num_events))
  current_page = 0
  current_event = 0
 
  fname = folder_name + ' plot_individual_events.pdf'
  with PdfPages(fname) as pdf:
    for page in range (num_pages):
      fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols)
      # print (axes.flat)
      for ax in axes.flat:
        if (current_event < num_events):
          event_time = df_list_of_apnea_events.iloc[current_event].at["Data Date-Time"]
          plot_start = event_time - start_offset
          plot_end = event_time + end_offset
          df_event_data = df[((df["Data Date-Time"] > plot_start) & (df["Data Date-Time"] < plot_end))]
          #print ("Current Event Index is " + str(current_event))
          current_event = current_event + 1
          plot_title = folder_name + " Event " + str(current_event) + ":  " + str(event_time)
          plotSPO2AndHR (df_event_data,plot_title,plot_start,plot_end,ax)
      print ("tight layout")
      fig.tight_layout()
      pdf.savefig()
      plt.close()

time: 5.54 ms (started: 2023-03-01 08:20:21 +00:00)


## **Main Function**

### Column Names

In [27]:
# Global Variables
### Column Names
Duration_Actual = "Event Duration (s) [Actual Data]"
Duration_Custom = "Event Duration (s) [Using Custom Settings - Theoretical Calculation]"
Duration_Preset = "Event Duration (s) [Using Preset Settings - Theoretical Calculation]"

Count_Actual = "Event Count [Actual Data]"
Count_Custom = "Event Count [Using Custom Settings - Theoretical Calculation]"
Count_Preset = "Event Count (s) [Using Preset Settings - Theoretical Calculation]"

SumDuration_Actual = "Cumulative Duration (s) [Actual Data]"
SumDuration_Custom = "Cumulative Duration (s) [Using Custom Settings - Theoretical Calculation]"
SumDuration_Preset = "Cumulative Duration (s) [Using Preset Settings - Theoretical Calculation]"

time: 1.94 ms (started: 2023-03-01 08:20:21 +00:00)


### Create Data Tables

In [28]:
#print (data_path)
df_LOGs = compileLogFiles (data_path)

df_s, df_d = splitToDataAndSettingsDF (df_LOGs)
os.chdir(analysis_path)
df_d.to_csv (folder_name + " - Data.xlsx")
#dataframe_to_pdf(df_d, folder_name + " - Data")

df_s.to_csv (folder_name + " - Settings.xlsx")
dataframe_to_pdf(df_s, folder_name + " - Settings")


df_j = pd.merge (df_d,df_s, on='Settings Index')
df_j = AddNumActualStimulations(df_j)
df_j = AnalyzeWithCustomThresholds (df_j)
df_j = AnalyzeWithPresetThresholds (df_j)
# display (df_j)
#df_j.to_csv (folder_name + " - Data and Settings Joined.xlsx")
#dataframe_to_pdf(df_j, folder_name + " - Data and Settings Joined")


/content/drive/MyDrive/Improve World Health/Document Control/NBD - New Business Development/NBD028 - ApneBoot/Clinical Data/St. John's Data/st. John's data_For Analysis/intervention/Baby of Geetha/Geetha
Number of Files: 4
['LOG00184 (1).TXT', 'LOG00185 (1).TXT', 'LOG00186 (1).TXT', 'LOG00187 (1).TXT']
14/07/19,10:06:54,000,000,3,0000,0,0,0,
Not able to Detect Nonin Nor Zug
unknown data code version


UnboundLocalError: ignored

time: 3.64 s (started: 2023-03-01 08:20:21 +00:00)


### Create Plots

In [ ]:
df_j.dtypes
plotByDay (df_j)
plotByHour (df_j)
PlotApneaEventsGrid (df_j)
#plotApneBootData(df_j)

### Create Data Summary Tables

In [ ]:
def createSummaryTables (df,df_s):
  
  df_s_columns_dropped = df_s.drop(['Settings Date','Settings Time','Alarm intensity'], axis=1)
  
  reportApneaCountBySetting (df,df_s_columns_dropped)
  reportIndividualActualApneas (df)

In [ ]:
createSummaryTables (df_j,df_s)

In [ ]:
# df_j stands for df_joined
# display (df_j[df_j["Preset Threshold - Apnea Duration (s)"] !=0])
#df["Time (hours)"]={len(df)*1/60/60)
# df_j.to_csv (folder_name + " All_Data_Joined_Table.csv")
# df_s.to_csv (folder_name + " Settings_Index_Table.csv")

In [ ]:
# display(df_j[df_j["Stimulation Triggered (Old Formula for Actual Settings)"]==1])
# display(df_j[df_j["Stimulation Triggered (New Formula for Actual Settings)"]==1])
# display(df_j[df_j["Stimulation Triggered (New Formula for Actual Settings)"]!= df_j["Stimulation Triggered (Old Formula for Actual Settings)"]])
# display(df_j[df_j["Preset Threshold - Apnea Duration (s)"]==0])

# DRAFT - New Section

In [ ]:
#cwd
#df=pd.read_csv("LOG00034.txt")

## ** Not Used Code**

In [ ]:
# #fname = "Report.pdf"
# with PdfPages(fname) as pdf:
#   print("This is a test of the print function")
#   pdf.savefig()

In [ ]:
# usage_pv_data_table = pd.pivot_table(data_table, index =['Data Date',"Settings Index"],
#                          aggfunc = 'count')

# print(usage_pv_data_table)
# usage_pv_data_table.to_csv('Usage_pv_data_table.csv',index=True)
# usage_pv_data_table.to_excel("Usage_pv_data_table.xlsx")





In [ ]:
pd.reset_option("all")
#df = stimulation_results_table_with_all_thresholds
#print(df)
#pivot = pd.pivot_table(stimulation_results_table_with_all_thresholds,
                       #values=["Status"], 
#                       index=['Status'])
                       #aggfunc={'Status': np.count,
                       # 'HR': [min, max, np.mean]})

#df = df.Status.value_counts()
#print(pivot)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option("precision", 4)

#print(df)


#print(len (stimulation_results_table_with_all_thresholds))
#filtered_results_table_all = df[(df["Status"]==129) | (df["Status"]==131)]
#print(len (filtered_results_table_all))

#data_table = filtered_results_table_all

# pivot = pd.pivot_table(data_table,
#                        values=['SPO2','HR'], 
#                        index=['Data Date'],
#                        aggfunc={'SPO2': np.mean,
#                         'HR': [min, max, np.mean]})

#print(pivot)
#filtered_results_table_all.to_csv("Filtered_Results_Table_All.csv")

# 129    91112
# 153    49477
# 131    32220



Want 
- Date
- Setting Index
- 



In [ ]:
# # !sudo apt-get install wkhtmltopdf
# # !yum install wkhtmltopdf
# !apt-get install wkhtmltopdf
# import pandas as pd 
# df_apnea_summary_table.to_html('HTMLTest.html')  

# import pdfkit 

# pdfkit.from_url('HTMLTest.html', 'HTMLTest.pdf')

In [ ]:
df = pd.DataFrame({'Col': [1,4,5,6,7],
                   'Col2': [1,4,5,6,7],
                   })
display (df)
New_Col = "Col3"
df [New_Col] = df["Col"] +1

display (df)
display (df[New_Col])

In [ ]:
# Create a pandas dataframe with demo data:
#!pip install -e git+https://github.com/mindee/doctr.git#egg=python-doctr[torch]
#!pip install weasyprint
!pip install django-weasyprint
import pandas as pd
demodata_csv = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
df = pd.read_csv(demodata_csv)

def dfToPDF (df):
  # Pretty print the dataframe as an html table to a file
  intermediate_html = '/tmp/intermediate.html'
  to_html_pretty(df,intermediate_html,'Iris Data')
  # if you do not want pretty printing, just use pandas:
  # df.to_html(intermediate_html)

  # Convert the html file to a pdf file using weasyprint
  import weasyprint
  out_pdf= '/tmp/demo.pdf'
  weasyprint.HTML(intermediate_html).write_pdf(out_pdf)

  # This is the table pretty printer used above:

def to_html_pretty(df, filename='/tmp/out.html', title=''):
    '''
    Write an entire dataframe to an HTML file
    with nice formatting.
    Thanks to @stackoverflowuser2010 for the
    pretty printer see https://stackoverflow.com/a/47723330/362951
    '''
    ht = ''
    if title != '':
        ht += '<h2> %s </h2>\n' % title
    ht += df.to_html(classes='wide', escape=False)

    with open(filename, 'w') as f:
         f.write(HTML_TEMPLATE1 + ht + HTML_TEMPLATE2)

HTML_TEMPLATE1 = '''
<html>
<head>
<style>
  h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 90%;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%; 
  }
</style>
</head>
<body>
'''

HTML_TEMPLATE2 = '''
</body>
</html>
'''

In [ ]:
# # PRINT TABLES & DATA
# from matplotlib.backends.backend_pdf import PdfPages
# #https://stackoverflow.com/questions/4042192/reduce-left-and-right-margins-in-matplotlib-plot
# #df_apnea_summary_table.plot()
# #dfToPDF (df)
# # fig = convertDataframeToFigure (apnea_summary_table)
# # pdf = PdfPages("foo.pdf")
# # pdf.savefig(fig, bbox_inches='tight')
# # pdf.close()
# # !pip install -c conda-forge python-pdfkit
# !pip3 install wkhtmltopdf
# !pip3 install pdfkit

# import pandas as pd
# import pdfkit as pdf
# import sqlite3

# # con=sqlite3.connect("baza.db")

# # df=pd.read_sql_query("select * from dobit", con)
# #df_apnea_summary_table.datetime = pd.to_datetime(apnea_summary_table.datetime)
# display(df_apnea_summary_table)

# df_apnea_summary_table.to_html('apnea_summary_table.html')
# nazivFajla='test.pdf'
# config = pdf.configuration(wkhtmltopdf=path_wkhtmltopdf)
# pdf.from_url('apnea_summary_table.html', nazivFajla, configuration=config)


